# TimeSmith: Getting Started

This notebook introduces the TimeSmith library and its core concepts.

## What is TimeSmith?

TimeSmith is a time series machine learning library with:
- **Strict layer boundaries**: Clean architecture with four distinct layers
- **Type safety**: Runtime validation of scientific types
- **Composition**: Flexible pipelines and adapters
- **Performance**: Optional Numba JIT compilation for speed

## Architecture Overview

1. **Typing Layer**: Scientific types (`SeriesLike`, `PanelLike`, `TableLike`, `ForecastLike`)
2. **Core Layer**: Base classes (`BaseTransformer`, `BaseForecaster`, `BaseDetector`, `BaseFeaturizer`)
3. **Compose Layer**: Pipelines, adapters, and feature unions
4. **Tasks & Eval Layer**: Task definitions and evaluation tools


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Import TimeSmith
import timesmith as ts

# Import plotting utilities (uses plotsmith)
try:
    from timesmith.utils.plotting import plot_timeseries, plot_forecast
    import plotsmith as ps
    HAS_PLOTTING = True
except ImportError:
    print("Warning: plotsmith not installed. Install with: pip install plotsmith")
    HAS_PLOTTING = False

print(f"TimeSmith version: {ts.__version__}")


## Scientific Types

TimeSmith uses runtime-validated scientific types to ensure data consistency:


In [ ]:
# Create a time series
dates = pd.date_range('2020-01-01', periods=100, freq='D')
values = np.random.randn(100).cumsum() + 100
y = pd.Series(values, index=dates, name='temperature')

# Check if it's a valid SeriesLike
print(f"Is SeriesLike: {ts.is_series(y)}")
print(f"Series shape: {y.shape}")
print(f"Index type: {type(y.index)}")

# Display first few values
y.head()


## Basic Forecasting Example

Let's create a simple forecasting pipeline:


In [ ]:
# Create a forecast task
task = ts.ForecastTask(y=y, fh=10, frequency='D')
print(f"Forecast horizon: {task.fh}")
print(f"Frequency: {task.frequency}")

# Create a simple moving average forecaster
forecaster = ts.SimpleMovingAverageForecaster(window=7)

# Fit and predict
forecaster.fit(y)
forecast = forecaster.predict(task.fh)

print(f"\nForecast shape: {forecast.predicted.shape}")
print(f"Forecast values:\n{forecast.predicted.head()}")


In [ ]:
# Visualize the forecast using plotsmith
if HAS_PLOTTING:
    fig, ax = plot_forecast(
        historical=y,
        forecast=forecast.predicted,
        intervals=forecast.intervals,
        title='Simple Moving Average Forecast'
    )
    import matplotlib.pyplot as plt
    plt.show()
else:
    print("Plotting not available. Install plotsmith: pip install plotsmith")


## Basic Transformation Example

TimeSmith provides many transformers for preprocessing:


In [ ]:
# Create data with outliers
y_with_outliers = y.copy()
y_with_outliers.iloc[20] = 150  # Add outlier
y_with_outliers.iloc[50] = -50  # Add outlier

# Remove outliers
outlier_remover = ts.OutlierRemover(method='iqr', factor=1.5)
y_clean = outlier_remover.fit_transform(y_with_outliers)

print(f"Original data points: {len(y_with_outliers)}")
print(f"Cleaned data points: {len(y_clean)}")
print(f"Outliers removed: {len(y_with_outliers) - len(y_clean)}")


## Next Steps

Continue to the next notebooks to learn about:
- Data loading and preprocessing
- Feature engineering
- Advanced forecasting
- Anomaly detection
- Network analysis
- Evaluation and backtesting
